In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [4]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [5]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [6]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [25]:
params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'degree': [2, 3, 4],
    'class_weight': [None, 'balanced']
}

svc = SVC(random_state = 2025)

In [26]:
np.random.seed(2025)
svc_grid = GridSearchCV(svc, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
svc_grid.fit(X_temp, y_temp)
print('Best Parameters:', svc_grid.best_params_)

Best Parameters: {'C': 10, 'class_weight': None, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf'}


In [29]:
np.random.seed(2025)
svc_model = SVC(C = 10, class_weight = None, degree = 2, gamma = 'auto', kernel = 'rbf', random_state = 2025)
svc_model.fit(X_temp, y_temp,)
y_pred = svc_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        36
           1       1.00      0.78      0.88         9

    accuracy                           0.96        45
   macro avg       0.97      0.89      0.92        45
weighted avg       0.96      0.96      0.95        45



In [31]:
np.random.seed(2025)
svc_model.fit(X_full, y)
y_pred_svc = svc_model.predict(X_test_full)

In [32]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_svc})

with open('SVC_Full_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [36]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [37]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [43]:
np.random.seed(2025)
svc_grid_rfe = GridSearchCV(svc, param_grid = params, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
svc_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', svc_grid_rfe.best_params_)

Best Parameters: {'C': 10, 'class_weight': None, 'degree': 2, 'gamma': 'auto', 'kernel': 'sigmoid'}


In [44]:
np.random.seed(2025)
svc_model_rfe = SVC(C = 10, class_weight = None, degree = 2, gamma = 'auto', kernel = 'sigmoid', random_state = 2025)
svc_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = svc_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe, zero_division = 1))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89        36
           1       1.00      0.00      0.00         9

    accuracy                           0.80        45
   macro avg       0.90      0.50      0.44        45
weighted avg       0.84      0.80      0.71        45



In [45]:
np.random.seed(2025)
svc_model_rfe.fit(X_RFE, y)
y_pred_rfe_svc = svc_model_rfe.predict(X_RFE_test)

In [46]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_svc})

with open('SVC_RFE_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [47]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [48]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [49]:
np.random.seed(2025)
svc_grid_lasso = GridSearchCV(svc, param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
svc_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', svc_grid_lasso.best_params_)

Best Parameters: {'C': 10, 'class_weight': 'balanced', 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}


In [50]:
np.random.seed(2025)
svc_model_lasso = SVC(C = 10, class_weight = 'balanced', degree = 2, gamma = 'scale', kernel = 'sigmoid', random_state = 2025)
svc_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = svc_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso,zero_division = 1))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74        36
           1       0.32      0.67      0.43         9

    accuracy                           0.64        45
   macro avg       0.60      0.65      0.59        45
weighted avg       0.77      0.64      0.68        45



In [51]:
np.random.seed(2025)
svc_model_lasso.fit(X_LASSO, y)
y_pred_lasso_svc = svc_model_lasso.predict(X_LASSO_test)

In [52]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_svc})

with open('SVC_LASSO_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [53]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [54]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [55]:
np.random.seed(2025)
svc_grid = GridSearchCV(svc, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
svc_grid.fit(X_temp, y_temp)
print('Best Parameters:', svc_grid.best_params_)

Best Parameters: {'C': 100, 'class_weight': None, 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}


In [56]:
np.random.seed(2025)
svc_model = SVC(C = 100, class_weight = None, degree = 2, gamma = 'scale', kernel = 'sigmoid', random_state = 2025)
svc_model.fit(X_temp, y_temp)
y_pred_svc_no = svc_model.predict(X_test)

print(classification_report(y_test, y_pred_svc_no))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84        36
           1       0.29      0.22      0.25         9

    accuracy                           0.73        45
   macro avg       0.55      0.54      0.54        45
weighted avg       0.71      0.73      0.72        45



In [57]:
np.random.seed(2025)
svc_model.fit(X_full_no, y)
y_pred_svc_no = svc_model.predict(X_test_full_no)

In [58]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_svc_no})

with open('SVC_Full_No_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [59]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [60]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [61]:
np.random.seed(2025)
svc_grid = GridSearchCV(svc, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
svc_grid.fit(X_temp, y_temp)
print('Best Parameters:', svc_grid.best_params_)

Best Parameters: {'C': 100, 'class_weight': 'balanced', 'degree': 2, 'gamma': 'scale', 'kernel': 'sigmoid'}


In [62]:
np.random.seed(2025)
svc_model = SVC(C = 100, class_weight = 'balanced', degree = 2, gamma = 'scale', kernel = 'sigmoid', random_state = 2025)
svc_model.fit(X_temp, y_temp)
y_pred_manual = svc_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       0.88      0.61      0.72        36
           1       0.30      0.67      0.41         9

    accuracy                           0.62        45
   macro avg       0.59      0.64      0.57        45
weighted avg       0.76      0.62      0.66        45



In [63]:
np.random.seed(2025)
svc_model.fit(X_no_manual, y)
y_pred_svc_no_manual = svc_model.predict(X_test_no_manual)

In [64]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_svc_no_manual})

with open('SVC_No_Manual_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [65]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [66]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [67]:
np.random.seed(2025)
svc_grid = GridSearchCV(svc, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
svc_grid.fit(X_temp, y_temp)
print('Best Parameters:', svc_grid.best_params_)

Best Parameters: {'C': 0.1, 'class_weight': 'balanced', 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}


In [68]:
np.random.seed(2025)
svc_model = SVC(C = 0.1, class_weight = 'balanced', degree = 2, gamma = 'scale', kernel = 'poly', random_state = 2025)
svc_model.fit(X_temp, y_temp)
y_pred_manual = svc_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       0.86      0.83      0.85        36
           1       0.40      0.44      0.42         9

    accuracy                           0.76        45
   macro avg       0.63      0.64      0.63        45
weighted avg       0.77      0.76      0.76        45



In [69]:
np.random.seed(2025)
svc_model.fit(X_manual, y)
y_pred_svc_manual = svc_model.predict(X_manual_test)

In [70]:
from google.colab import files

pred_svc = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_svc_manual})

with open('SVC_Manual_Cov.csv', 'w') as file:
    pred_svc.to_csv(file, index = False, header = True)

files.download('SVC_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GHOST:


In [71]:
!pip install ghostml

In [72]:
import ghostml
from sklearn import metrics

def calc_metrics(labels_test, test_probs, threshold = 0.5):
  scores = [1 if x>=threshold else 0 for x in test_probs]
  auc = metrics.roc_auc_score(labels_test, test_probs)
  kappa = metrics.cohen_kappa_score(labels_test, scores)
  confusion = metrics.confusion_matrix(labels_test, scores, labels = list(set(labels_test)))

  print('Threshold: %.2f, kappa: %.3f, AUC test set: %.3f'%(threshold, kappa, auc))
  print(confusion)
  print(metrics.classification_report(labels_test, scores))

  return

thresholds = np.round(np.arange(0.01, 0.75,0.01),2)

In [110]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

svc_ghost = SVC(C = 100, class_weight = 'balanced', degree = 2, gamma = 'scale', kernel = 'sigmoid', probability = True, random_state = 2025)
svc_ghost.fit(X_temp, y_temp)
svc_test_probs = svc_ghost.predict_proba(X_test)[:,1]
calc_metrics(y_test, svc_test_probs, threshold = 0.5)

Threshold: 0.50, kappa: 0.000, AUC test set: 0.586
[[36  0]
 [ 9  0]]
              precision    recall  f1-score   support

           0       0.80      1.00      0.89        36
           1       0.00      0.00      0.00         9

    accuracy                           0.80        45
   macro avg       0.40      0.50      0.44        45
weighted avg       0.64      0.80      0.71        45



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [111]:
np.random.seed(2025)
svc_temp_probs = svc_ghost.predict_proba(X_temp)[:,1]
svc_threshold = ghostml.optimize_threshold_from_predictions(y_temp, svc_temp_probs, thresholds, ThOpt_metrics = 'Kappa')

np.random.seed(2025)
calc_metrics(y_test, svc_test_probs, threshold = svc_threshold)
svc_GHOST_test = [1 if x >= svc_threshold else 0 for x in svc_test_probs]

Threshold: 0.21, kappa: 0.190, AUC test set: 0.586
[[22 14]
 [ 3  6]]
              precision    recall  f1-score   support

           0       0.88      0.61      0.72        36
           1       0.30      0.67      0.41         9

    accuracy                           0.62        45
   macro avg       0.59      0.64      0.57        45
weighted avg       0.76      0.62      0.66        45



In [112]:
np.random.seed(2025)
svc_ghost.fit(X_full, y)
svc_probs = svc_ghost.predict_proba(X_test_full)[:,1]
svc_pred_ghost = [1 if x >= svc_threshold else 0 for x in svc_probs]

In [113]:
from google.colab import files

pred_rf = pd.DataFrame({'ID': range(1, 92), 'Placement': svc_pred_ghost})

with open('SVC_GHOST_Cov.csv', 'w') as file:
    pred_rf.to_csv(file, index = False, header = True)

files.download('SVC_GHOST_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>